In [37]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

decimal_math_template = """You are a mathematician. You have expertize in decimal arithmatic operations. 
You breakdown complex problem in steps and solve one step at a time. Please show detailed steps when solving. 
If you are asked anything other than decimal arithmatic operation then just say that you don't know how to solve it.
Here is a question for you to solve: {input}"""

binary_math_template = """You are a mathematician. You have expertize in binary arithmatic operations. 
You breakdown complex problem in steps and solve one step at a time. Please show detailed steps when solving.
If you are asked anything other than binary arithmatic operation then just say that you don't know how to solve it.
Here is a question for you to solve: {input}"""

hexadecimal_math_template = """You are a mathematician. You have expertize in hexadecimal arithmatic operations. 
You breakdown complex problem in steps and solve one step at a time. Please show detailed steps when solving.
If you are asked anything other than hexadecimal arithmatic operation then just say that you don't know how to solve it.
Here is a question for you to solve: {input}"""

prompt_info_array = [
    {
        'name': 'decimal',
        'description': 'Mathematician for solving decimal digits problems',
        'prompt_template': decimal_math_template
    },
    {
        'name': 'binary',
        'description': 'Mathematician for solving binary digits problems',
        'prompt_template': binary_math_template
    },
    {
        'name': 'hexadecimal',
        'description': 'Mathematician for solving hexadecimal digits problems',
        'prompt_template': hexadecimal_math_template
    }
]

llm = OpenAI()

destination_chains = {}
for p_info in prompt_info_array:
    name = p_info['name']
    prompt_template = p_info['prompt_template']
    prompt = PromptTemplate(template=prompt_template, input_variables=['input'])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    
default_chain = ConversationChain(llm=llm, output_key="text")

In [38]:
# LLMRouterChain example. The chain execution order is not hardcoded. LLM is invoked to decide the next chanin to execute

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain


destinations = [f"{p['name']}: {p['description']}" for p in prompt_info_array]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
print(chain.run("What is twenty four divided by six?"))
print(chain.run("What is the output of addition of binary numbers 001100 and 1100000?"))
print(chain.run("What is the result of adition of A34C3D and 100?"))
print(chain.run("What is the result of 1100 or 0110?"))
print(chain.run("Tell me soemthing about Mars."))

In [35]:
# Embeddings router chain example. Uses embeddings of description instead of router to decide the next chain to execute

from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

name_and_desc = []
for p in prompt_info_array:
    name_and_desc.append((p["name"], [p["description"]]))

embedding_router_chain = EmbeddingRouterChain.from_names_and_descriptions(name_and_desc, Chroma, OpenAIEmbeddings(), routing_keys=["input"])

e_chain = MultiPromptChain(
    router_chain=embedding_router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)
print(name_and_desc)

[('decimal', ['Mathematician for solving decimal digits problems']), ('binary', ['Mathematician for solving binary digits problems']), ('hexadecimal', ['Mathematician for solving hexadecimal digits problems'])]


In [36]:
print(e_chain.run("What is twenty four divided by six?"))
print(e_chain.run("What is the output of addition of binary numbers 001100 and 1100000?"))
print(e_chain.run("What is the result of adition of A34C3D and 100?"))
print(e_chain.run("What is the result of 1100 or 0110?"))
print(e_chain.run("Tell me soemthing about Mars."))



> Entering new MultiPromptChain chain...
hexadecimal: {'input': 'Tell me soemthing about Mars.'}
> Finished chain.

I'm sorry, I don't know how to solve this.
